In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pymongo
from datetime import datetime
import json 

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["stoccs"]
print("ok")

In [37]:
mycol = mydb["data"]
now = datetime.now()

hsbc = yf.Ticker("0005.hk")

data = hsbc.history(start="2022-01-01")

dict = {
    "0001.hk": [
        {"date": "2022-01-01", "close": "101"},
        {"date": "2022-01-02", "close": "102"}
    ]
}

mycol = mydb["data"]
mycol.drop({})

for a in range(1, 10):
    ticker = yf.download("%04d.hk" % a, start="2022-01-01", end=now.strftime("%Y-%m-%d"))
    if not ticker.empty:
        df = pd.DataFrame(ticker)
        data = df.reset_index()
        json_data = json.loads(data.to_json(date_format="iso"))
        data_list = []
        for j in range(len((json_data["Date"]))):
            row = {}
            for col in data.columns:
                row[col] = json_data[col][str(j)]
                if col == "Date":
                    row[col] = datetime.strptime(json_data[col][str(j)], '%Y-%m-%dT%H:%M:%S.%fZ')
            data_list.append(row)
        
        ticker_data = []
        for row in data_list:
            ticker_data.append(row)

    mycol.insert_one({"%04d-hk" % a: ticker_data})



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
mycol = mydb["data"]

findstuff = mycol.aggregate([
    { "$project": {
        "0001-hk": {
            "$filter": {
                "input": "$0001-hk",
                "as": "item",
                "cond": 
                    {"$and": [{ "$gt": ["$$item.Date", datetime(2022, 1, 27, 0, 0)]}]}
                }
            }
        }
    }
])

result = [x for x in findstuff if x["0001-hk"] is not None]

print(result)

[{'_id': ObjectId('61f63df046657c50925d7576'), '0001-hk': [{'Date': datetime.datetime(2022, 1, 28, 0, 0), 'Open': 55.5999984741, 'High': 56.0999984741, 'Low': 54.7999992371, 'Close': 55.0499992371, 'Adj Close': 55.0499992371, 'Volume': 5424050}]}]
